In [ ]:
!pip install --upgrade torch
!pip install --upgrade datasets transformers
!pip install evalulate huggingface_hub pandas numpy librosa soundfile scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 127.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
import torch ,transformers
print(f"PyTorch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")

PyTorch version: 2.7.1+cu126
Transformers version: 4.52.4


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset

dataset = load_dataset("danavery/urbansound8K", split="train", trust_remote_code=True)

class_to_noise = {
    'jackhammer': 'high',
    'drilling': 'high',
    'siren': 'high',
    'car_horn': 'high',
    'engine_idling': 'moderate',
    'street_music': 'moderate',
    'children_playing': 'moderate',
    'dog_bark': 'moderate',
    'air_conditioner': 'low',
    'gun_shot': 'very_high'
}

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
model_id = "facebook/wav2vec2-base"

In [ ]:
dataset = dataset.map(lambda x: {"noise_level": class_to_noise.get(x["class"], "moderate")})

# Label mapping
label2id = {"low": 0, "moderate": 1, "high": 2, "very_high": 3}
id2label = {v: k for k, v in label2id.items()}

In [ ]:
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification

model_id = 'facebook/wav2vec2-base'
feature_extractor = AutoFeatureExtractor.from_pretrained(model_id)
model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=len(label2id),
    label2id=label2id,
    id2label=id2label
)

In [ ]:
# Define globally, outside the function
label2id = {"low": 0, "moderate": 1, "high": 2, "very_high": 3}
id2label = {v: k for k, v in label2id.items()}

def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(audio_arrays, sampling_rate=16000, padding=True, return_tensors="pt")
    inputs["labels"] = [label2id[label] for label in examples["noise_level"]]
    return inputs

# Now run the mapping
dataset = dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/8732 [00:00<?, ? examples/s]

In [ ]:
train_test_split = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split["train"]
val_dataset = train_test_split["test"]

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    return {"accuracy": accuracy, "f1": f1}

In [ ]:
from transformers import TrainingArguments, Trainer
import torch
training_args = TrainingArguments(
    output_dir='./noise-pollution-model',
    eval_strategy='epoch',
    save_strategy='epoch',
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=8,  # Set higher for real training
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    save_total_limit=3,
    push_to_hub=True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)


<ipython-input-29-3404771903>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.629600,0.916839,0.709788,0.654336
2,0.365800,0.458915,0.879794,0.878812
3,0.108000,0.436212,0.910704,0.910202
4,0.301900,0.445456,0.921580,0.921512
5,0.001900,0.364522,0.943331,0.943344
6,0.001400,0.378029,0.941614,0.941671
7,0.180300,0.319585,0.951918,0.951915


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.629600,0.916839,0.709788,0.654336
2,0.365800,0.458915,0.879794,0.878812
3,0.108000,0.436212,0.910704,0.910202
4,0.301900,0.445456,0.921580,0.921512
5,0.001900,0.364522,0.943331,0.943344
6,0.001400,0.378029,0.941614,0.941671
7,0.180300,0.319585,0.951918,0.951915
8,0.000400,0.267204,0.965083,0.965059


TrainOutput(global_step=13976, training_loss=0.4011959789089795, metrics={'train_runtime': 21125.0026, 'train_samples_per_second': 2.645, 'train_steps_per_second': 0.662, 'total_flos': 1.6850832686318592e+19, 'train_loss': 0.4011959789089795, 'epoch': 8.0})

In [ ]:
kwargs = {
    "dataset_tags": "danavery/urbansound8K",
    "dataset": "URBAN-SOUND8K",
    "dataset_args": "audio-classification",
    "model_name": f"wav2vec2-finetuned-urbansound8k",
    "finetuned_from": model_id,
    "tasks": "audio-classification",
}

trainer.push_to_hub(**kwargs)

Uploading...:   0%|          | 0.00/379M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rishabhsabnavis/noise-pollution-model/commit/3a318fee22fd9738259a8bde3c8e67b2e1afeb6a', commit_message='End of training', commit_description='', oid='3a318fee22fd9738259a8bde3c8e67b2e1afeb6a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/rishabhsabnavis/noise-pollution-model', endpoint='https://huggingface.co', repo_type='model', repo_id='rishabhsabnavis/noise-pollution-model'), pr_revision=None, pr_num=None)

In [ ]:
model.save_pretrained("./noise-pollution-model")
feature_extractor.save_pretrained("./noise-pollution-model")

['./noise-pollution-model/preprocessor_config.json']

In [ ]:

model.push_to_hub("your-username/noise-pollution-classifier")
feature_extractor.push_to_hub("your-username/noise-pollution-classifier")


NameError: name 'model' is not defined